In [ ]:
import numpy as np

# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar_allsub'
sensor = 'lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [253]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample'
sensor = 'lidar'
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --num_process {num_process}'

/data/kimgh/CenterPoint-custom/CenterPoint-static


'!python create_data.py nia_data_prep --root_path /data/kimgh/CenterPoint-custom/CenterPoint-static/data/sample --sensor lidar --num_process 20'

# # Train
---
- pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_lidar" --no-autorestart -- 3 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/lidar_allsub/train --validate

- CUDA_VISIBLE_DEVICES=0,1,2,3 pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_radar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/radar_allsub/train --validate

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = './result/lidar/train'

!python train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py'
work_dir = './result/lidar_allsub/train'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/lidar/train'

!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [ ]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar/normal_allsubid'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_lidar.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=1 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test' 

In [ ]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/lidar/abnormal_allsubid'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_lidar.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=2 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [ ]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar/normal'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_radar.pth'
sensor = 'radar'

!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test

In [ ]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/radar/abnormal'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/weights/nia_centerpoint_static_radar.pth'

!CUDA_VISIBLE_DEVICES=3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --speed_test